Experiment with OpenCV for finding the pool table edges within an image.

This version assumes a full-sized vertical image of the table. It is looking
for the marker pattern along the side of the table.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
import copy

In [ ]:
def ImgShow(images, resolution=120):
    plt.rcParams['figure.dpi'] = resolution
    
    cnt = len(images)
    if cnt > 20:
        # Top-level object is an image, make it an array of 1
        images = [images]
        cnt = len(images)
        
    cols = cnt
    fig, ax = plt.subplots(1, cnt)
    if 1 == cnt:
        ax = [ax]
    for i in range(0, len(images)):
        dims = len(images[i].shape)
        if 3 == dims:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_BGR2RGB)
        else:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_GRAY2RGB)
            
        ax[i].axis('off')
        ax[i].imshow(imgDisp)
        


def ColorRange(clr, fuzz):    
    colorMin = (max(0,clr[0]-fuzz), max(0,clr[1]-fuzz), max(0,clr[2]-fuzz))    
    colorMax = (min(255,clr[0]+fuzz), min(255,clr[1]+fuzz), min(255,clr[2]+fuzz))
    return colorMin, colorMax
        
#Handy color switcher
clrs = []
clrs.append((255,0,0))
clrs.append((0,255,0))
clrs.append((0,0,255))
clrs.append((255,255,0))
clrs.append((255,0,255))
clrs.append((0,255,255))
iClr = 0

def Color():
    global iClr
    iClr += 1
    return clrs[iClr % len(clrs)]
    
clr = Color()

In [ ]:
# Handy circle class
class Circle:
    def __init__(self, x,y,r):
        self.x = x
        self.y = y
        self.r = r
        
    def center(self):
        return (self.x, self.y)
    
    def point(self):
        return (int(self.x+0.5), int(self.y+0.5))
    
    def radius(self):
        return self.r

# Handy line class
class LineP:
    def __init__(self, circleA, circleB):
        x1 = circleA.center()[0]
        y1 = circleA.center()[1]
        x2 = circleB.center()[0]
        y2 = circleB.center()[1]
        rise = y2 - y1
        run = x2 - x1
        if run != 0.0:
            self.m = rise/run
        else:
            self.m = 10E10
        self.b = y1 - self.m * x1
        self.radius = (circleA.radius() + circleB.radius()) / 2.0
        self.circles = []
        self.circles.append(circleA)
        self.circles.append(circleB)
    
    def addCircle(self, circle):
        # scan for duplicates
        for c in self.circles:
            if c.radius() == circle.radius() and c.center()[0] == circle.center()[0] and c.center()[1] == circle.center()[1]:
                return
        self.circles.append(circle)
        
    def val(self):
        return self.m, self.b
        
    def angleRad(self):
        return np.arctan(self.m)
    
    def angleDeg(self):
        return self.angleRad() * 180.0 / math.pi
    
    def pointCount(self):
        return len(self.circles)
    
    def circleRadius(self):
        return self.radius
        
    def distance(self, point):
        # This doesn't work right
        a = self.m
        b = -1
        c = self.b        
        x = point[0]
        y = point[1]
        d = abs(a*x + b*y + c) / math.sqrt(a*a + b*b)
        return d
    
    def endpoints(self):
        # Scan for the farthest points
        d2Max = 0.0
        for i in range(0, len(self.circles)-1):
            for j in range(i+1, len(self.circles)):
                dx = self.circles[i].center()[0] - self.circles[j].center()[0]
                dy = self.circles[i].center()[1] - self.circles[j].center()[1]
                d2 = dx*dx + dy*dy
                if d2 > d2Max:
                    d2Max = d2
                    p1 = self.circles[i].point()
                    p2 = self.circles[j].point()
        return p1,p2
    
    def length(self):
        # distance between endpoints
        p1,p2 = self.endpoints()
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        return math.sqrt(dx*dx + dy*dy)
    
    
# Test the endpoints
c1 = Circle(0.0,0.0,1.0)
c2 = Circle(10.0,0.0,1.0)
c3 = Circle(20.0,0.0,1.0)
line = LineP(c1, c2)
line.addCircle(c1)
line.addCircle(c3)
line.addCircle(c2)
pt1, pt2 = line.endpoints()
print(pt1, pt2)
m,b = line.val()
print(m,b)

In [ ]:
# Talk to the amcrest camera
from amcrest import AmcrestCamera
import os

if False:
    camera = AmcrestCamera('192.168.0.209', 80, 'honeybadger', 'DoucheBag').camera
    #print(camera.software_information)
    filename = "__amcrest_camera_.jpg"
    camera.snapshot(path_file=filename)
    imageCam = cv.imread(filename)
    os.remove(filename)
    ImgShow(imageCam)


In [ ]:
# Load the test image
imageOriginal = cv.imread("../test/images/long_all.jpg")

In [ ]:
# Hough Circles. 
# Limit by size range and distance
cols = imageOriginal.shape[1]
rows = imageOriginal.shape[0]
imageGray = cv.cvtColor(imageOriginal, cv.COLOR_BGR2GRAY)
imageGray = cv.medianBlur(imageGray, 5)
hcircles = cv.HoughCircles(imageGray, cv.HOUGH_GRADIENT, minDist=5, dp=1,
                               param1=100, param2=10,
                               minRadius=3, maxRadius=13)

# Create an array of our custom Circle objects to hold
circles = []
imageCircles = imageOriginal.copy()
if hcircles is not None:
    hcircles = np.uint16(np.around(hcircles))
    for i in hcircles[0, :]:        
        c = Circle(i[0], i[1], i[2])
    
        # Filter out points ouside of image
        pt = c.point()
        if pt[0] < 0 or pt[1] < 0:
            continue
        if pt[0] > cols-1 or pt[1] > rows-1:
            continue
    
        circles.append(c)
        cv.circle(imageCircles, pt, c.radius(), Color(), 7)
        
print("circles count = {0}".format(len(circles)))
    
ImgShow([imageGray, imageCircles], 220)

In [ ]:
# Filter out circles that don't match the color we want
# We will assume that the circle centers should be whiteish

# Find the range
clrWhite = (240, 240, 240)
fuzz = 35

imageCirclesWhite = imageOriginal.copy()
imageCirclesWhiteBlack = np.zeros(imageOriginal.shape[:3],np.uint8)

clrMin, clrMax = ColorRange(clrWhite, fuzz)
clrMin = (clrMin[0]-1,clrMin[1]-1,clrMin[2]-1,)
clrMax = (clrMax[0]+1,clrMax[1]+1,clrMax[2]+1,)
print("color range = {0} - {1}".format(clrMin, clrMax))
circlesWhite = []
for c in circles:
    pt = c.point()
    
    clr = imageOriginal[pt[1],pt[0]]
    if (clr > clrMin).all() and (clr < clrMax).all():
        circlesWhite.append(c)
        clr = Color()
        cv.circle(imageCirclesWhite, c.point(), c.radius(), clr, 6)
        cv.circle(imageCirclesWhiteBlack, c.point(), c.radius(), clr, 6)
        #print("pt={0} clr={1}".format(pt, clr))
print("Prunage: {0} - {1}".format(len(circles), len(circlesWhite)))

ImgShow([imageCirclesWhite, imageCirclesWhiteBlack], 220)

In [ ]:
# Find the edges using flood 
cols = imageOriginal.shape[1]
rows = imageOriginal.shape[0]
seed = (int(cols / 2), int(rows / 2)) # a point in the middle

imageFlood = imageOriginal.copy()

# Make an empty black image to accept the mask
imageFloodMask = np.zeros((rows+2,cols+2,1), np.uint8)
tolerance = 70.0
diff = (tolerance,tolerance,tolerance)
flags = 8 | cv.FLOODFILL_FIXED_RANGE | 255 << 8
res = cv.floodFill(imageFlood, imageFloodMask, seed, (0,0,255),diff,diff, flags)
imageFloodMask = imageFloodMask[1:rows+1,1:cols+1] # Remove border from mask so that it perfectly matches the original

cannyThresh = 100.0
imageCanny = cv.Canny(imageFloodMask, cannyThresh, cannyThresh * 2)

ImgShow([imageFlood, imageFloodMask, imageCanny],200)

In [ ]:
# Use GrabCut to find outer table region

imageTable = imageOriginal.copy()

cols = imageTable.shape[1]
rows = imageTable.shape[0]

imageTableMask = np.zeros(imageTable.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
border = 20
rect = (border,border,cols-border,rows-border)
cv.grabCut(imageTable,imageTableMask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT)
mask2 = np.where((imageTableMask==2)|(imageTableMask==0),0,1).astype('uint8')
imageTable = imageTable*mask2[:,:,np.newaxis]


ImgShow([imageTable, imageTableMask, mask2])

In [ ]:
import cv2

cols = imageOriginal.shape[1]
rows = imageOriginal.shape[0]

img = imageOriginal.copy()
mask = np.zeros(img.shape[:2],np.uint8)   # img.shape[:2] = (413, 620)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (0,0,cols,rows)

# this modifies mask 
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)

# If mask==2 or mask== 1, mask2 get 0, other wise it gets 1 as 'uint8' type.
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')

# adding additional dimension for rgb to the mask, by default it gets 1
# multiply it with input image to get the segmented image
img_cut = img*mask2[:,:,np.newaxis]


ImgShow([img,mask])

In [ ]:
# Find the table using CutGrab algorithm
imageCutGrabMask = np.zeros(imageOriginal.shape[:2],np.uint8)

ImgShow([imageCutGrabMask])

In [ ]:
# Find the outside of the table flood
contoursFelt, hierarchy = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

imageFeltContours = imageOriginal.copy()
imageFeltHull = imageOriginal.copy()

# combine into a single array of points
#listPoints = contoursFelt[0]
listPoints = []
for i in range(len(contoursFelt)):
    # creating convex hull object for each contour
    cnt = contoursFelt[i]
    for pt in cnt:
        listPoints.append(pt)    

# create hull array for convex hull points
hull = cv.convexHull(cnt, False)
for pt in hull:
    print(pt[0],pt[1])

# create an empty black image
imageFeltContours = np.zeros((imageOriginal.shape[0], imageOriginal.shape[1], 3), np.uint8)
imageFeltHull = np.zeros((imageOriginal.shape[0], imageOriginal.shape[1], 3), np.uint8)

# draw contours and hull points
color_contours = (0, 255, 0) # green - color for contours
# draw ith contour
cv.drawContours(imageFeltContours, contoursFelt, -1, color_contours, 4, 8, hierarchy)

# draw ith convex hull object
cv.drawContours(imageFeltHull, hull, -1, color_contours, 4, 8)

ImgShow([imageFeltContours, imageFeltHull], 200)

In [ ]:
# Find the outside of the table floodvector<vector<Point> > contours;
contoursFelt, b = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for i in range(len(contoursFelt)):
    color_contours = (0, 255, 0) # green - color for contours
    color = (255, 0, 0) # blue - color for convex hull
    # draw ith contour
    cv2.drawContours(drawing, contoursFelt, i, color_contours, 1, 8, hierarchy)

    # draw ith convex hull object

    cv2.drawContours(drawing, hull, i, color, 1, 8)


# Merge all contours into a single array
listAllContourPoints = []
for i in range(len(contoursFelt)):
    for j in range(len(contoursFelt[i])):
        listAllContourPoints.append(contoursFelt[i][j])        
        if(len(listAllContourPoints) > 5):
            break
    if(len(listAllContourPoints) > 5):
        break
        
listHull = cv.convexHull(listAllContourPoints)
    
imageFeltContours = imageOriginal.copy()
imageFeltHull = imageOriginal.copy()

for i in range(len(contoursFelt)):
    cv.drawContours(imageFeltContours, contoursFelt, i, Color())
    cv.drawContours(imageFeltHull, listHull, i, Color())

ImgShow([imageFeltContours, imageFeltHull], 200)

In [ ]:
# Threshold out by marker color
ptMarkers = [
    (435,1231),
    (2653, 1958)
]

def ColorRange(clr, fuzz):    
    colorMin = copy.deepcopy(clr)
    colorMin[0] = max(0,clr[0]-fuzz)
    colorMin[1] = max(0,clr[1]-fuzz)
    colorMin[2] = max(0,clr[2]-fuzz)
    
    colorMax = copy.deepcopy(clr)
    colorMax[0] = min(255,clr[0]+fuzz)
    colorMax[1] = min(255,clr[1]+fuzz)
    colorMax[2] = min(255,clr[2]+fuzz)
    
    return colorMin, colorMax

ptMarker = ptMarkers[0]

# Look at the color at that pixel
fuzz = 33
color = imageOriginal[ptMarker[1],ptMarker[0]]
colorMin, colorMax = ColorRange(color,fuzz)
imageMarkerMask = cv.inRange(imageOriginal, colorMin, colorMax)

ImgShow([imageOriginal, imageMarkerMask])

In [ ]:
# Find the edges, start with a blur
blur = int(5)
imageBlur = cv.GaussianBlur(imageMarkerMask, (blur,blur), cv.BORDER_DEFAULT)

# Edge detect using canny
if True:
    imageCanny = cv.Canny(imageBlur, 100, 175)
else:
    imageCannyMarksOnly = cv.imread("../test/images/CannyMarksOnly.jpg")
    imageCanny = cv.Canny(imageCannyMarksOnly, 100, 175)

ImgShow([imageBlur, imageCanny],220)

In [ ]:
# Contours
contours, b, = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
imgDrawnContours = imageOriginal.copy()
cv.drawContours(imgDrawnContours, contours, -1, (0,255,0),3)
print("Contours = ", len(contours))
ImgShow([imageCanny,imgDrawnContours], 220)

In [ ]:
# Find circles    
circleRadiusMin = 5
circleRadiusMax = 35

circles = []
for points in contours:
    (x,y),radius = cv.minEnclosingCircle(points)
    
    # Skip circles that are too big or too small
    if radius > circleRadiusMax or radius < circleRadiusMin:
        continue
    
    circles.append(Circle(x,y,radius))
    
#sort by radius
circles.sort(key=lambda s: s.radius()) # sorts using lambda function

# print out the data
imgCircles = imageOriginal.copy()
for circle in circles:
    #print("{0}\t{1}\t{2}".format(circle.radius(), circle.center()[0], circle.center()[1]))
    center_coordinates = (int(circle.center()[0]+0.5), int(circle.center()[1]+0.5))
    cv.circle(imgCircles, center_coordinates, int(circle.radius()), Color(), 3)	
    
print("Circle Count = ", len(circles))
ImgShow([imgCircles], 200)
#cv.imwrite("../test/images/dotz.jpg", imgCircles)

In [ ]:
# Between every two points, there is a line. Find all of the possible lines.
linesAll = []
imgLinesAll = imageOriginal.copy()
maxRadiusDiff = 6.0
minCircleRadius = 3.0
for i in range(0, len(circles)-1):
    for j in range(i+1, len(circles)):
        circleA = circles[i]
        circleB = circles[j]
        
        # Skip circles that are too small
        if circleB.radius() < minCircleRadius or circleA.radius() < minCircleRadius:
            continue
        
        # Don't create lines between circles of different sizes
        radiusDiff = np.abs(circleB.radius() - circleA.radius())
        if(radiusDiff > maxRadiusDiff):
            continue   
        linesAll.append(LineP(circleA, circleB))
        cv.line(imgLinesAll, circleA.point(), circleB.point(), Color(), 3)

# Add each circle to all lines that it lies upon
for circle in circles:
    # Find the line this might fit on. Give it to all lines that match.
    for line in linesAll:
        if line.distance(circle.center()) > maxRadiusDiff/2.0:
            continue
        radDiff = abs(circle.radius() - line.circleRadius())
        line.addCircle(circle)
        if(radDiff <= maxRadiusDiff):
            line.addCircle(circle)  # Close to the line and withing radius tolerance
            
print("All Lines ", len(linesAll))
ImgShow([imgLinesAll],150)    


In [ ]:
# Extract vertical lines

imgLinesVertical = imageOriginal.copy()
angleLimitV = 14.0 # Only accept lines +/- this angle
linesVertical = []
for line in linesAll:        
    angle = line.angleDeg()
    #print(angle)
    if(angle > 90-angleLimitV and angle < 90+angleLimitV or angle > -90-angleLimitV and angle < -90+angleLimitV):
        linesVertical.append(line)
        p1,p2 = line.endpoints()
        cv.line(imgLinesVertical, p1,p2, Color(), 3)

print("Vertical Lines ", len(linesVertical))

ImgShow([imgLinesVertical],150)    


In [ ]:
# Extract only the lines that have enough circles that land on it
imgLinesPruned = imageOriginal.copy()
linesPruned= []
# Grab the lines with more than 3 points
for line in linesVertical:
    if line.pointCount() >= 5:
        linesPruned.append(line)
        p1,p2 = line.endpoints()
        cv.line(imgLinesPruned, p1,p2, Color(), 4)
        
print("Pruned Lines len=",len(linesPruned))
        
ImgShow([imgLinesPruned], 150)
for line in linesPruned:
    m,b = line.val()
    #print("line: ", m, b)

In [ ]:
# Find the leftmost and rightmost lines
rows = imageOriginal.shape[0]
cols = imageOriginal.shape[1]
ymid = rows / 2.0
minLeft = cols
maxRight = 0
for line in linesPruned:
    m,b = line.val()
    xmid = (ymid-b)/m
    
    # Left-most?
    if(xmid < minLeft):
        minLeft = xmid
        lineLeft = line
    if(xmid > maxRight):
        maxRight = xmid
        lineRight = line
        
imgLinesSides = imageOriginal.copy()
p1,p2 = lineLeft.endpoints()
cv.line(imgLinesSides, p1,p2, Color(), 4)
p1,p2 = lineRight.endpoints()
cv.line(imgLinesSides, p1,p2, Color(), 4)
ImgShow([imgLinesSides], 150)

In [ ]:
# Extract horizontal lines

imgLinesHorizontal = imageOriginal.copy()
angleLimitH = 5.0 # Only accept lines +/- this angle
linesHorizontal = []
for line in linesAll:        
    angle = line.angleDeg()
    if(angle > -angleLimitH and angle < angleLimitH):
        linesHorizontal.append(line)
        p1,p2 = line.endpoints()
        cv.line(imgLinesHorizontal, p1,p2, Color(), 3)

print("Horizontal Lines ", len(linesHorizontal))

ImgShow([imgLinesHorizontal],150)    


In [ ]:
# Find the topmost and bottommost lines
rows = imageOriginal.shape[0]
cols = imageOriginal.shape[1]
xmid = cols / 2.0
minTop = rows
maxBottom = 0
for line in linesHorizontal:
    m,b = line.val()
    ymid = m*xmid + b
    
    # Left-most?
    if(ymid < minTop):
        minTop = ymid
        lineTop = line
    if(ymid > maxBottom):
        maxBottom = ymid
        lineBottom = line
        
imgLinesTopBottom = imageOriginal.copy()

p1,p2 = lineTop.endpoints()
cv.line(imgLinesTopBottom, p1,p2, Color(), 4)
p1,p2 = lineBottom.endpoints()
cv.line(imgLinesTopBottom, p1,p2, Color(), 4)
ImgShow([imgLinesTopBottom], 150)

In [ ]:
# Show all 4 sides
imgLinesEdges = imageOriginal.copy()

p1,p2 = lineLeft.endpoints()
cv.line(imgLinesEdges, p1,p2, Color(), 4)
p1,p2 = lineRight.endpoints()
cv.line(imgLinesEdges, p1,p2, Color(), 4)

p1,p2 = lineTop.endpoints()
cv.line(imgLinesEdges, p1,p2, Color(), 4)
p1,p2 = lineBottom.endpoints()
cv.line(imgLinesEdges, p1,p2, Color(), 4)

ImgShow([imgLinesEdges], 150)